In [ ]:
#Imports
import torch
import torchvision
import torch.nn.functional as F
import time
import cv2
import numpy as np
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot


In [ ]:
# Create a model object that uses alexnet as a constructor for image classification
# Alexnet is a CNN 
# We will not use a pretrained version although 
#https://pytorch.org/docs/stable/generated/torch.nn.Linear.html
model = torchvision.models.alexnet(pretrained=False)
#Applies a linear transformation to the data 
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

In [ ]:
#This will load in the pytorch model created in the training in the train_model section
# load deserializes the data 
# the learnable parameters are weights and biases 
# We have convolutional layers
model.load_state_dict(torch.load('best_model.pth'))

In [ ]:
#The device to load our tensor (vector) into memory
# Basically moving the model weights and biases from the CPU to GPU
# so we can look at high resolution vid/imgs
device = torch.device('cuda')
model = model.to(device)

In [ ]:
#This section will make the model that we trained match the camera

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])
#
normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    """
    camera_value: image received from the neural network
    return x: a suitable ipnut to the neural network
    """
    global device, normalize
    x = camera_value
    #Convert BGR to RGB
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [ ]:

#Create our camera object and widgets to show the camera image and control the speed
camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
speed_slider = widgets.FloatSlider(description='speed', min=0.0, max=0.5, value=0.0, step=0.01, orientation='horizontal')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.VBox([widgets.HBox([image, blocked_slider]), speed_slider]))

In [ ]:
#Create robot object
robot = Robot()

In [ ]:
#Supervised Logistic learning (Classification)
#Using Alexnet CNN
def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it 
    #sums to 1 (which makes it a probability distribution)
    # Basically zi = (xi – min(x)) / (max(x) – min(x)) where z is the normalized value
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.5:
        robot.left(-speed_slider.value)
    else:
        robot.forward(-speed_slider.value)
    
    time.sleep(0.001)
        
update({'new': camera.value})  #

In [ ]:
#Attatches the each frame of the camera to the update fcn
camera.observe(update, names='value')

In [ ]:
camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

In [ ]:
camera.stop()